In [1]:
# from dspy.ai

import dspy
from dspy.datasets import HotPotQA
from transformers import AutoTokenizer, AutoModel

# Configure DSPy to use a Hugging Face model
dspy.configure(lm=dspy.LM('bert-base-uncased'))

# Define a search function using Hugging Face's tokenizer and model
def search(query: str) -> list[str]:
    tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
    model = AutoModel.from_pretrained('bert-base-uncased')
    inputs = tokenizer(query, return_tensors='pt')
    outputs = model(**inputs)
        # This is a placeholder for actual search logic
        return ["Result 1", "Result 2", "Result 3"]

# Set up a RAG module
class RAG(dspy.Module):
    def __init__(self):
        self.respond = dspy.ChainOfThought('context, question -> response')

    def forward(self, question):
        context = search(question)
        return self.respond(context=context, question=question)

# Instantiate and use the RAG module
rag = RAG()
response = rag(question="What is the capital of France?")
print(response)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BadRequestError: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=bert-base-uncased
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers

In [3]:
import dspy
import pandas as pd
from transformers import AutoTokenizer, AutoModel

# Load the CSV file
csv_data = pd.read_csv('151_ideas_updated2.csv')

# Configure DSPy to use a Hugging Face model
dspy.configure(lm=dspy.LM('bert-base-uncased'))

# Define a search function using Hugging Face's tokenizer and model
def search(query: str) -> list[str]:
    tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
    model = AutoModel.from_pretrained('bert-base-uncased')
    inputs = tokenizer(query, return_tensors='pt')
    outputs = model(**inputs)
    # This is a placeholder for actual search logic
    return ["Result 1", "Result 2", "Result 3"]

# Set up a RAG module
class RAG(dspy.Module):
    def __init__(self):
        self.respond = dspy.ChainOfThought('context, question -> response')

    def forward(self, question):
        context = search(question)
        return self.respond(context=context, question=question)

# Instantiate and use the RAG module
rag = RAG()

# Example usage with data from the CSV
for index, row in csv_data.iterrows():
    question = row['Ideas']  # Replace with the actual column name
    response = rag(question=question)
    print(response)


BadRequestError: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=bert-base-uncased
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers

In [11]:
import dspy
import pandas as pd
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM
from typing import Literal, Optional

# --- HFModel Class ---
def openai_to_hf(**kwargs):
    hf_kwargs = {}
    for k, v in kwargs.items():
        if k == "n":
            hf_kwargs["num_return_sequences"] = v
        elif k == "frequency_penalty":
            hf_kwargs["repetition_penalty"] = 1.0 - v
        elif k == "presence_penalty":
            hf_kwargs["diversity_penalty"] = v
        elif k == "max_tokens":
            hf_kwargs["max_new_tokens"] = v
        elif k == "model":
            pass
        else:
            hf_kwargs[k] = v
    return hf_kwargs

class HFModel(dspy.LM):
    def __init__(self, model: str, checkpoint: Optional[str] = None, is_client: bool = False,
                 hf_device_map: Literal["auto", "balanced", "balanced_low_0", "sequential"] = "auto"):
        super().__init__(model)
        self.provider = "hf"
        self.is_client = is_client
        self.device_map = hf_device_map
        if not self.is_client:
            try:
                import torch
                from transformers import AutoConfig, AutoModelForCausalLM, AutoModelForSeq2SeqLM, AutoTokenizer
            except ImportError as exc:
                raise ModuleNotFoundError(
                    "You need to install Hugging Face transformers library to use HF models.",
                ) from exc
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            try:
                architecture = AutoConfig.from_pretrained(model).__dict__["architectures"][0]
                self.encoder_decoder_model = ("ConditionalGeneration" in architecture) or ("T5WithLMHeadModel" in architecture)
                self.decoder_only_model = ("CausalLM" in architecture) or ("GPT2LMHeadModel" in architecture)
                assert self.encoder_decoder_model or self.decoder_only_model, f"Unknown HuggingFace model class: {model}"
                self.tokenizer = AutoTokenizer.from_pretrained(model if checkpoint is None else checkpoint)

                self.rationale = True
                AutoModelClass = AutoModelForSeq2SeqLM if self.encoder_decoder_model else AutoModelForCausalLM
                if checkpoint:
                    self.rationale = False
                    if self.device_map:
                        self.model = AutoModelClass.from_pretrained(checkpoint, device_map=self.device_map)
                    else:
                        self.model = AutoModelClass.from_pretrained(checkpoint).to(self.device)
                else:
                    if self.device_map:
                        self.model = AutoModelClass.from_pretrained(model, device_map=self.device_map)
                    else:
                        self.model = AutoModelClass.from_pretrained(model).to(self.device)
                self.drop_prompt_from_output = False
            except ValueError:
                self.model = AutoModelForCausalLM.from_pretrained(
                    model if checkpoint is None else checkpoint,
                    device_map=self.device_map,
                )
                self.drop_prompt_from_output = True
                self.tokenizer = AutoTokenizer.from_pretrained(model)
                self.drop_prompt_from_output = True
        self.history = []

    def basic_request(self, prompt, **kwargs):
        raw_kwargs = kwargs
        kwargs = {**self.kwargs, **kwargs}
        response = self._generate(prompt, **kwargs)

        history = {
            "prompt": prompt,
            "response": response,
            "kwargs": kwargs,
            "raw_kwargs": raw_kwargs,
        }
        self.history.append(history)

        return response

    def _generate(self, prompt, **kwargs):
        assert not self.is_client
        kwargs = {**openai_to_hf(**self.kwargs), **openai_to_hf(**kwargs)}
        if isinstance(prompt, dict):
            try:
                prompt = prompt['messages'][0]['content']
            except (KeyError, IndexError, TypeError):
                print("Failed to extract 'content' from the prompt.")
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)

        outputs = self.model.generate(**inputs, **kwargs)
        if self.drop_prompt_from_output:
            input_length = inputs.input_ids.shape[1]
            outputs = outputs[:, input_length:]
        completions = [
            {"text": c}
            for c in self.tokenizer.batch_decode(outputs, skip_special_tokens=True)
        ]
        response = {
            "prompt": prompt,
            "choices": completions,
        }
        return response

    def __call__(self, prompt, only_completed=True, return_sorted=False, **kwargs):
        assert only_completed, "for now"
        assert return_sorted is False, "for now"

        if kwargs.get("n", 1) > 1 or kwargs.get("temperature", 0.0) > 0.1:
            kwargs["do_sample"] = True

        response = self.request(prompt, **kwargs)
        return [c["text"] for c in response["choices"]]
# --- End HFModel Class ---

# Load the CSV file
csv_data = pd.read_csv('151_ideas_updated2.csv')

# Configure DSPy to use a smaller Hugging Face model (using HFModel)
dspy.configure(lm=HFModel('t5-small'))

# Define a search function using Distilbert
def search(query: str) -> list[str]:
    tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
    model = AutoModel.from_pretrained('distilbert-base-uncased')
    inputs = tokenizer(query, return_tensors='pt')
    outputs = model(**inputs)
    return [f"Search result for: {query} (using DistilBERT embeddings placeholder)"]

# Set up a RAG module
class RAG(dspy.Module):
    def __init__(self):
        self.respond = dspy.ChainOfThought('context, question -> response')

    def forward(self, question):
        context = search(question)
        return self.respond(context=context, question=question)

# Instantiate and use the RAG module
rag = RAG()

# Example usage with data from the CSV
for index, row in csv_data.iterrows():
    question = row['Ideas']
    response = rag(question=question)
    print(response)

TypeError: HFModel.__call__() missing 1 required positional argument: 'prompt'